# MIRCF Results Reproduction — Section 6

Deterministic reproduction of all empirical results reported in Section 6.

- One Research Question per section
- No mutable global state
- CI-safe execution


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import sem
import os, warnings

warnings.filterwarnings('ignore')
plt.rcParams.update({'figure.dpi': 300, 'font.size': 11})

EXPORT_DIR = 'outputs'
FIG_DPI = 300
os.makedirs(EXPORT_DIR, exist_ok=True)

In [ ]:
def abel_regularize(x, eps):
    return x / (1 + eps * np.abs(x))

def EDB_scalar(x, sigma=0.1):
    p = 0.5 * (1 + np.tanh(x / sigma))
    p = np.clip(p, 1e-9, 1 - p)
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

def spectral_gap(A):
    vals = np.sort(np.real_if_close(np.linalg.eigvals(A)))[::-1]
    return vals[0] - vals[1] if len(vals) > 1 else 0.0

def row_normalize(A):
    rs = A.sum(axis=1, keepdims=True)
    rs[rs == 0] = 1.0
    return A / rs

In [ ]:
def generate_adj(N, mode, p=0.1, d=4, seed=0):
    rng = np.random.default_rng(seed)
    A = np.zeros((N, N))
    if mode == 'ring':
        for i in range(N):
            A[i,(i+1)%N] = A[i,(i-1)%N] = 1
    elif mode == 'erdos_renyi':
        A = (rng.random((N,N)) < p).astype(float)
        A = np.triu(A,1); A += A.T
    elif mode == 'fully_connected':
        A[:] = 1; np.fill_diagonal(A,0)
    elif mode == 'ramanujan_bipartite':
        N1 = N//2
        for i in range(N1):
            for j in rng.choice(range(N1,N), size=d, replace=False):
                A[i,j] = A[j,i] = 1
    return A

In [ ]:
def mircf_step(x_t, x_tm1, a, b, eps, K_RSI, K_EDB, gamma, W):
    x_raw = a*x_t + b*x_tm1
    x_reg = abel_regularize(x_raw, eps)
    rsi = np.exp(-0.1*np.abs(x_reg-x_t))
    edb = np.array([EDB_scalar(v) for v in x_reg])
    u = -K_RSI*np.maximum(0,0.5-rsi)*x_reg - K_EDB*edb*np.sign(x_reg)
    return x_reg + u + gamma*(W@x_t-x_t), rsi, edb

def run_mircf(cfg, seed):
    rng = np.random.default_rng(seed)
    A = generate_adj(cfg['N'], cfg['mode'], cfg.get('p',0), cfg.get('d',4), seed)
    W = row_normalize(A)
    X = np.zeros((cfg['T'], cfg['N']))
    RSI = np.zeros_like(X)
    EDB = np.zeros_like(X)
    V = np.zeros(cfg['T'])
    X[0] = rng.normal(0,0.6,cfg['N']); X[1] = rng.normal(0,0.6,cfg['N'])
    for t in range(1,cfg['T']-1):
        X[t+1], RSI[t+1], EDB[t+1] = mircf_step(X[t],X[t-1],cfg['a'],cfg['b'],cfg['eps'],cfg['K_RSI'],cfg['K_EDB'],cfg['gamma'],W)
        V[t+1] = 0.5*np.sum((X[t+1]-X[t+1].mean())**2)
    return dict(X=X,RSI=RSI,EDB=EDB,V=V,A=A)